<a href="https://colab.research.google.com/github/rohanjain1648/UDHBANI-HACKATHON/blob/main/visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dash
!pip install geopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [3]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import numpy as np
import flask

# Load dataset
df = pd.read_csv("/content/PS3-dataset.csv")
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Drop rows with NaN values in important columns
df = df.dropna(subset=['Date', 'Traffic Volume', 'Congestion Level'])

# Flask Server
server = flask.Flask(__name__)
app = dash.Dash(__name__, server=server, suppress_callback_exceptions=True)

# Layout
app.layout = html.Div([
    html.H1("Real-Time Traffic Dashboard", style={'textAlign': 'center', 'color': '#333'}),
    html.Div([
        dcc.Dropdown(
            id='area-dropdown',
            options=[{'label': area, 'value': area} for area in df['Area Name'].unique()],
            value=df['Area Name'].unique()[0] if len(df['Area Name'].unique()) > 0 else None,
            clearable=False,
            style={'width': '50%', 'margin': 'auto'}
        )
    ], style={'textAlign': 'center', 'marginBottom': '20px'}),
    html.Div([
        dcc.Graph(id='traffic-volume-graph'),
        dcc.Graph(id='congestion-level-heatmap')
    ], style={'display': 'flex', 'flexDirection': 'column', 'alignItems': 'center'})
])




In [4]:
# Callbacks
@app.callback(
    Output('traffic-volume-graph', 'figure'),
    Input('area-dropdown', 'value')
)
def update_traffic_graph(selected_area):
    filtered_df = df[df['Area Name'] == selected_area]
    if filtered_df.empty:
        return px.line(title=f'No Data Available for {selected_area}')
    fig = px.line(filtered_df, x='Date', y='Traffic Volume', title=f'Traffic Volume in {selected_area}',
                  labels={'Date': 'Date', 'Traffic Volume': 'Volume'}, template='plotly_dark')
    return fig

@app.callback(
    Output('congestion-level-heatmap', 'figure'),
    Input('area-dropdown', 'value')
)
def update_congestion_map(selected_area):
    filtered_df = df[df['Area Name'] == selected_area]
    if filtered_df.empty:
        return px.scatter(title=f'No Data Available for {selected_area}')
    fig = px.scatter(filtered_df, x='Road/Intersection Name', y='Congestion Level', color='Congestion Level',
                     size='Traffic Volume', title=f'Congestion Levels in {selected_area}',
                     labels={'Road/Intersection Name': 'Road', 'Congestion Level': 'Level'}, template='plotly_dark')
    return fig

# Run Server
if __name__ == '__main__':
    app.run_server(debug=True, host='0.0.0.0', port=8050)

<IPython.core.display.Javascript object>